In [126]:
# !pip install catboost

In [127]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from imblearn.under_sampling import *
from imblearn.over_sampling import *
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import *
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import *
from xgboost import XGBClassifier
from sklearn.feature_selection import *
from catboost import CatBoostClassifier
import lightgbm as lgb


df = pd.read_csv('company_train.csv').drop(["ID", "retained_earnings"], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62896 entries, 0 to 62895
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   company_name                   62896 non-null  object 
 1   status_label                   62896 non-null  object 
 2   year                           62896 non-null  int64  
 3   cost_of_goods_sold             62896 non-null  float64
 4   depreciation_and_amortization  62896 non-null  float64
 5   ebitda                         62896 non-null  float64
 6   inventory                      62896 non-null  float64
 7   total_receivables              62896 non-null  float64
 8   market_value                   62896 non-null  float64
 9   net_sales                      62896 non-null  float64
 10  total_assets                   62896 non-null  float64
 11  total_long_term_debt           62896 non-null  float64
 12  ebit                           62896 non-null 

In [128]:
df['net_profit_margin_category'] = df['net_profit_margin_category'].map({'Low Profit Margin': 0, 'Healthy Profit Margin': 1, 'High Profit Margin': 2})
df['status_label'] = df['status_label'].map({'alive': 1, 'failed': 0})

In [129]:

# Define a function to create new features
def create_new_features(df):
    # Avoid division by zero by replacing zeros with a small number
    df['net_sales'].replace(0, 1e-6, inplace=True)
    df['total_assets'].replace(0, 1e-6, inplace=True)
    df['total_assets_minus_liabilities'] = df['total_assets'] - df['total_liabilities']
    df['total_assets_minus_liabilities'].replace(0, 1e-6, inplace=True)
    df['total_receivables'].replace(0, 1e-6, inplace=True)

    # Gross Profit Margin
    df['gross_profit_margin'] = df['gross_profit'] / df['net_sales']

    # EBITDA Margin
    df['ebitda_margin'] = df['ebitda'] / df['net_sales']

    # EBIT Margin
    df['ebit_margin'] = df['ebit'] / df['net_sales']

    # Asset Turnover
    df['asset_turnover'] = df['net_sales'] / df['total_assets']

    # Debt-to-Equity Ratio
    df['debt_to_equity'] = df['total_liabilities'] / df['total_assets_minus_liabilities']

    # Receivables Turnover
    df['receivables_turnover'] = df['net_sales'] / df['total_receivables']

    # Operating Expense Ratio
    df['operating_expense_ratio'] = df['total_operating_expenses'] / df['net_sales']

    # Depreciation and Amortization Ratio
    df['depreciation_amortization_ratio'] = df['depreciation_and_amortization'] / df['total_assets']

    # Drop intermediate columns if not needed
    df.drop(['total_assets_minus_liabilities'], axis=1, inplace=True)

    return df

In [130]:
# Create new features in the training data
df = create_new_features(df)

# anchor1 = "gross_profit"
# anchor2 = "market_value"
# anchor3 = "gross_profit"
# anchor4 = "total_long_term_debt"

# rolling_n = 3
# df[f'{anchor1}_moving_avg'] = df.groupby('company_name')[anchor1].transform(lambda x: x.rolling(rolling_n, 1).mean())
# df[f'{anchor2}_moving_avg'] = df.groupby('company_name')[anchor2].transform(lambda x: x.rolling(rolling_n, 1).mean())
# df[f'{anchor3}_moving_avg'] = df.groupby('company_name')[anchor3].transform(lambda x: x.rolling(rolling_n, 1).mean())
# df[f'{anchor4}_moving_avg'] = df.groupby('company_name')[anchor4].transform(lambda x: x.rolling(rolling_n, 1).mean())
df.drop('company_name', axis=1, inplace=True)

<ipython-input-129-78608b0ffaf3>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['net_sales'].replace(0, 1e-6, inplace=True)
<ipython-input-129-78608b0ffaf3>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [131]:
print(len(df[df["net_profit_margin_category"] == 0].index))
print(len(df[df["net_profit_margin_category"] == 1].index))
print(len(df[df["net_profit_margin_category"] == 2].index))


42240
10210
10446


In [132]:
# oversample = RandomOverSampler(random_state=42)

# X, y = oversample.fit_resample(df.drop(["net_profit_margin_category"], axis=1), df["net_profit_margin_category"])

# df = pd.concat([X, y], axis=1)

In [133]:
print(len(df[df["net_profit_margin_category"] == 0].index))
print(len(df[df["net_profit_margin_category"] == 1].index))
print(len(df[df["net_profit_margin_category"] == 2].index))

42240
10210
10446


In [134]:
# no_year_all_numerics = df.drop(["year", "net_profit_margin_category"], axis=1)

# corr = no_year_all_numerics.corr()
# sns.heatmap(corr,
#             xticklabels=corr.columns.values,
#             yticklabels=corr.columns.values)

In [135]:
df

,status_label,year,cost_of_goods_sold,depreciation_and_amortization,ebitda,inventory,total_receivables,market_value,net_sales,total_assets,...,current_ratio,net_profit_margin_category,gross_profit_margin,ebitda_margin,ebit_margin,asset_turnover,debt_to_equity,receivables_turnover,operating_expense_ratio,depreciation_amortization_ratio
0,1,1999,833.107,18.373,89.031,336.018,128.348,372.7519,1024.333,740.998,...,3.12,0,0.186683,0.086916,0.068980,1.382369,1.182519,7.980903,0.913084,0.024795
1,1,2000,713.811,18.577,64.367,320.590,115.187,377.1180,874.255,701.854,...,3.87,0,0.183521,0.073625,0.052376,1.245637,1.062990,7.589876,0.926375,0.026468
2,1,2001,526.477,22.496,27.207,286.588,77.528,364.5928,638.721,710.199,...,2.90,0,0.175732,0.042596,0.007376,0.899355,1.289229,8.238585,0.957404,0.031676
3,1,2002,496.747,27.172,30.745,259.954,66.322,143.3295,606.337,686.621,...,1.95,0,0.180741,0.050706,0.005893,0.883074,1.327623,9.142321,0.949294,0.039574
4,1,2003,523.302,26.680,47.491,247.245,104.661,308.9071,651.958,709.292,...,3.29,0,0.197338,0.072844,0.031921,0.919167,1.351109,6.229235,0.927156,0.037615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62891,1,2011,44889.000,931.000,5143.000,5499.000,4537.000,18497.8566,51035.000,22839.000,...,2.28,0,0.120427,0.100774,0.082532,2.234555,1.145111,11.248622,0.899226,0.040764
62892,1,2012,38753.000,983.000,5583.000,5075.000,4209.000,32839.1385,45352.000,24220.000,...,2.41,1,0.145506,0.123104,0.101429,1.872502,1.166562,10.775006,0.876896,0.040586
62893,1,2013,36944.000,1021.000,6098.000,5279.000,4501.000,44059.5907,44062.000,27298.000,...,2.65,1,0.161545,0.138396,0.115224,1.614111,1.181397,9.789380,0.861604,0.037402
62894,1,2013,2.051,0.441,-34.415,7.271,0.660,52.0572,2.255,69.325,...,10.52,0,0.090466,-15.261641,-15.457206,0.032528,0.111245,3.416667,16.261641,0.006361


In [136]:
X = pd.get_dummies(
    df.drop(columns=[ 'net_profit_margin_category']),
    drop_first=True
)
y = df["net_profit_margin_category"]

In [137]:
# kbest = SelectKBest(score_func=f_classif, k=15)

# kbest.fit_transform(X, y)

# X = X[kbest.get_feature_names_out()]

In [138]:
# kbest.get_feature_names_out()

In [139]:
# knn = KNeighborsClassifier()

# gs3 = GridSearchCV(knn, param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'metric': ['euclidean', 'manhattan']}, cv=5, scoring='f1_macro', n_jobs=-1)

# gs_knn = gs3.fit(X, y)

# display(gs_knn.best_params_)
# display(gs_knn.best_score_)
# best_knn = gs_knn.best_estimator_
# best_knn

In [140]:
# xgb = XGBClassifier()

# gs2 = GridSearchCV(xgb, param_grid={'n_estimators': [100, 110, 125, 130, 140, 150], 'max_depth': [12, 13, 14, 15, 16, 17, 18, 19, 20]}, cv=5, scoring='f1_macro', n_jobs=-1)

# gs_xgb = gs2.fit(X, y)

# display(gs_xgb.best_params_)
# display(gs_xgb.best_score_)
# best_xgb = gs_xgb.best_estimator_
# best_xgb

In [141]:
# paramscb = {
#     'iterations': 100,
#     'learning_rate': 1,
#     'num_leaves': 31,
#     'class_weights': {0: 1, 1: 1.25, 2: 1.25},
# }

# cbc = CatBoostClassifier(**paramscb, verbose=0)

In [142]:
# params = res['params']

# display([params])

# cbc = CatBoostClassifier(**params, verbose=0)

In [143]:
rfc = RandomForestClassifier(
    n_estimators=135,
    max_depth=20,
class_weight= {0: 1, 1: 1.25, 2: 1.25},
)

In [144]:
# paramslg = {
#     'objective': 'multiclass',
#     'num_class': 3,
#     'metric': 'multi_logloss',
#     'learning_rate': 0.1,
#     'num_leaves': 40,
#     'max_depth': -1,
#     'class_weight': {0: 1, 1: 1.25, 2: 1.25},
# }

# lgbm = lgb.LGBMClassifier(**paramslg, verbose=0)

In [145]:
# etc = ExtraTreesClassifier(
#   random_state= 42
# )

In [146]:
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import cross_val_score

# skf = StratifiedKFold(n_splits=5)

# display(cross_val_score(etc, X, y, cv=skf, scoring='accuracy', verbose=3).mean())
# display(cross_val_score(etc, X, y, cv=skf, scoring='f1_macro', verbose=3).mean())

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

skf = StratifiedKFold(n_splits=5)

display(cross_val_score(rfc, X, y, cv=skf, scoring='accuracy', verbose=3).mean())
display(cross_val_score(rfc, X, y, cv=skf, scoring='f1_macro', verbose=3).mean())

[CV] END ................................ score: (test=0.838) total time=  50.7s
[CV] END ................................ score: (test=0.856) total time=  47.8s
[CV] END ................................ score: (test=0.861) total time=  46.0s
[CV] END ................................ score: (test=0.853) total time=  47.0s
[CV] END ................................ score: (test=0.855) total time=  46.1s


0.8528207584978673

[CV] END ................................ score: (test=0.772) total time=  46.5s
[CV] END ................................ score: (test=0.787) total time=  45.9s
[CV] END ................................ score: (test=0.781) total time=  46.6s
[CV] END ................................ score: (test=0.764) total time=  46.3s


In [ ]:
# test = pd.read_csv('company_test_classif.csv')

# test_x_ori = test.drop(["ID"], axis=1)
# id = test["ID"]

# test_x = test_x_ori.copy()
# test_x['status_label'] = test_x_ori['status_label'].map({'alive': 1, 'failed': 0})
# test_x = create_new_features(test_x)
# # test_x[f'{anchor1}_moving_avg'] = test_x.groupby('company_name')[anchor1].transform(lambda x: x.rolling(rolling_n, 1).mean())
# # test_x[f'{anchor2}_moving_avg'] = test_x.groupby('company_name')[anchor2].transform(lambda x: x.rolling(rolling_n, 1).mean())
# # test_x[f'{anchor3}_moving_avg'] = test_x.groupby('company_name')[anchor3].transform(lambda x: x.rolling(rolling_n, 1).mean())
# # test_x[f'{anchor4}_moving_avg'] = test_x.groupby('company_name')[anchor4].transform(lambda x: x.rolling(rolling_n, 1).mean())
# test_x.drop('company_name', axis=1, inplace=True)

# etc.fit(X, y)

# # y = pd.Series(lgbm.predict(test_x)).map({0: 'Low Profit Margin', 1: 'Healthy Profit Margin', 2: 'High Profit Margin'})
# y = pd.Series(etc.predict(test_x)).map({0: 'Low Profit Margin', 1: 'Healthy Profit Margin', 2: 'High Profit Margin'})

# prediction = pd.concat([id, y], axis=1).rename(columns={0: 'net_profit_margin_category'})

# display(prediction)

# prediction.to_csv('submit_etc_pesol_2.csv', index=False)

In [ ]:
test = pd.read_csv('company_test_classif.csv')

test_x_ori = test.drop(["ID"], axis=1)
id = test["ID"]

test_x = test_x_ori.copy()
test_x['status_label'] = test_x_ori['status_label'].map({'alive': 1, 'failed': 0})
test_x = create_new_features(test_x)
# test_x[f'{anchor1}_moving_avg'] = test_x.groupby('company_name')[anchor1].transform(lambda x: x.rolling(rolling_n, 1).mean())
# test_x[f'{anchor2}_moving_avg'] = test_x.groupby('company_name')[anchor2].transform(lambda x: x.rolling(rolling_n, 1).mean())
# test_x[f'{anchor3}_moving_avg'] = test_x.groupby('company_name')[anchor3].transform(lambda x: x.rolling(rolling_n, 1).mean())
# test_x[f'{anchor4}_moving_avg'] = test_x.groupby('company_name')[anchor4].transform(lambda x: x.rolling(rolling_n, 1).mean())
test_x.drop('company_name', axis=1, inplace=True)

rfc.fit(X, y)

# y = pd.Series(rfc.predict(test_x)).map({0: 'Low Profit Margin', 1: 'Healthy Profit Margin', 2: 'High Profit Margin'})
y = pd.Series(rfc.predict(test_x)).map({0: 'Low Profit Margin', 1: 'Healthy Profit Margin', 2: 'High Profit Margin'})

prediction = pd.concat([id, y], axis=1).rename(columns={0: 'net_profit_margin_category'})

display(prediction)

prediction.to_csv('submit_rfc_pesol_2.csv', index=False)

In [ ]:
test_x.head()

In [ ]:
# s1 = pd.read_csv('submit.csv')
# s2 = pd.read_csv('85.csv')

# s1.compare(s2)